# simbMoments

*simbMoments* determines a system of equations corresponding to the first and second moments of the population observations. The process to find each moment is quite similar to the way it was done to find the system of differential equations using *simbODE*. The equations are sympy objects which one can manipulate to compute some statistics from the whole population. The implemented function just determines upon the second moment. The first moment is equivalent to the mean behavior of the system output, while the second expresses the cross dependence of the species and is equivalent to the variance of the system output.

In [12]:
#libraries required
import numpy as np
import sympy as sp

**Defines System Properties**

In [13]:
#molecular species
species = ['x1', 'x2']
species = sp.var(species)

#system input
inp = ['u']
uh = sp.var(inp)
ruh = 0  #reaction affected by input (0 = 1st reaction)

#reagent and product matrices
reactants = np.array([[0, 1, 1, 0],
                      [0, 0, 0, 1]])
products = np.array([[1, 0, 1, 0],
                     [0, 0, 1, 0]])

#kinetic parameters
pars = ['c1', 'c2', 'c3', 'c4']
parsValues = sp.var(pars)
#to replace kinetic parameters for numeric values, uncomment the next line
#and comment the two previous ones
#parsValues = [4.0, 0.010, 1.0, 0.006]

**Pre-processing of the System**. From previous defined information determines stoichiometric matrix and propensity vector. These arrays are used to compute 
the moments of the system.

In [14]:
#stoichiometric matrix
V = products - reactants

#pre-propensity function
aPro = parsValues
aPro[ruh] = aPro[ruh]*uh[0]  #Attaches system input

#system dimentions
Sn, Rm = V.shape

#propensity function vector
for r in range(0,Rm):
    for s in range(0, Sn):
        #determines propensity vector expressions
        for a in range(0, reactants[s,r]):
            aPro[r] *= species[s]
        #end for a
    #end for s
#end for r
print("Stoichimotric Matrix:\n", V)
print("Propensity Function Vector:", aPro)

Stoichimotric Matrix:
 [[ 1 -1  0  0]
 [ 0  0  1 -1]]
Propensity Function Vector: [c1*u, c2*x1, c3*x1, c4*x2]


**Computes Moments Equations**. Each species has its own first moment equation and second orden moment equation. The second moment is found for itself and crossed with other species.

*First Moment*

In [15]:
#System of First Moment Equations
odeX = []
for s in range(0,Sn):
    temp = 0
    #Determines Defferential Equations
    for r in range(0, Rm):
        temp += V[s,r]*aPro[r]
    #end for r
    #Set of Differential Equations
    odeX.append(temp)
#end for s

*Second Order Moment*

In [16]:
#System of Second Order Equations
ode2m = []
name2m = []
nameODE = species
odeTotal = odeX
for s1 in range(0, Sn):
    for s2 in range(0, Sn):
        #Determines second order expression
        temp = 0
        for r in range(0,Rm):
            temp += (V[s1,r]*aPro[r]*species[s2] + V[s2,r]*aPro[r]*species[s1] \
                     + V[s1,r]*V[s2,r]*aPro[r])
        #end for r

        #set of second order moment equations
        if temp not in ode2m:
            ode2m.append(temp)
            odeTotal.append(temp)
        #end if temp

        #variable names of second order species
        if (species[s1]*species[s2]) not in name2m:
            name2m.append(species[s1]*species[s2])
            nameODE.append(species[s1]*species[s2])
        #end if species s1*s2
    #end for s2
#end for s1

Some processing of the determined data to make easy posterior manipulation

In [17]:
#replaces variable names
dxName = []
dxODE = []
for exp in odeTotal:
    #at each expression searches for the variable names to replaces them
    #for a nickname "dx#"
    for j in range(0,len(nameODE)):
        name = sp.var('dx' + str(len(nameODE)-j))
        exp = exp.subs(nameODE[len(nameODE)-j-1],name)
        
        #stores nicknames
        if name not in dxName:
            dxName.append(name)
        #end if name
    #end for j
    dxODE.append(exp)
#end for exp
dxName.reverse()

In [18]:
#Shows sistem of differential equations determined
for k in range(0,len(odeTotal)):
    print(f'd{nameODE[k]}/dt:', odeTotal[k])
 #     print(f'd({dxName[k]})/dt:', dxODE[k])
 #     print('\n')
 #end for k

dx1/dt: c1*u - c2*x1
dx2/dt: c3*x1 - c4*x2
dx1**2/dt: 2*c1*u*x1 + c1*u - 2*c2*x1**2 + c2*x1
dx1*x2/dt: c1*u*x2 - c2*x1*x2 + c3*x1**2 - c4*x1*x2
dx2**2/dt: 2*c3*x1*x2 + c3*x1 - 2*c4*x2**2 + c4*x2
